In [65]:
# https://www.bitaddress.org/
# https://github.com/pointbiz/bitaddress.org
# https://en.wikipedia.org/wiki/Base58

BASE58_CHARS = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

import binascii

def map_base58():
    '''Create a map of chars for Base58'''
    tmp_dir = {}
    i = 0
    for c in BASE58_CHARS:
        tmp_dir[c] = i
        i = i + 1
    return tmp_dir


BASE58_MAP = map_base58()


def base58_enc(in_num):
    '''Encode a number into Bitcoin Base58 format'''
    if in_num < 0:
        raise Exception("Positive integers only")
    tmp_out = []
    while in_num > 0:
        tmp_out.append(BASE58_CHARS[in_num % 58])
        in_num = in_num // 58
    tmp_out.reverse()
    return ''.join(tmp_out)


def base58_dec(in_str):
    '''Decode a Bitcoin Base58 string to a number'''
    out_num = 0
    for c in in_str:
        if c not in BASE58_MAP:
            raise Exception("Invalid char, not in Base58: %c" % c)
        out_num = out_num*58 + BASE58_MAP[c]
    return out_num


def base256_to_base58(raw_str):
    '''Convert a raw string to Base58 string'''
    leading_zeros = 0
    num = 0
    for c in raw_str:
        if c != 0:
            break
        leading_zeros = leading_zeros + 1
    for c in raw_str:
        num = num*256 + c
    base58_str = ''.join(leading_zeros*['1']) + base58_enc(num)
    return base58_str


def base58_to_base256(base58_str):
    '''Convert a Base58 string to raw string'''
    leading_ones = 0
    for c in base58_str:
        if c != '1':
            break
        leading_ones = leading_ones + 1
    num = base58_dec(base58_str)
    raw_str_suffix_hex = '%02x' % num
    if len(raw_str_suffix_hex) % 2:
        raw_str_suffix_hex = '0%s' % raw_str_suffix_hex
    raw_str_hex = ''.join(leading_ones*['00']) + raw_str_suffix_hex
    raw_str = binascii.unhexlify(raw_str_hex)
    return raw_str

In [66]:
import hashlib, binascii


def hash160(in_bytes):
    '''Performs RIPEMD160(SHA256(in_bytes)) and returns raw digest'''
    sha256_hasher = hashlib.sha256(in_bytes)
    ripemd160_hasher = hashlib.new('ripemd160')
    ripemd160_hasher.update(sha256_hasher.digest())
    return ripemd160_hasher.digest()


def hash256(in_bytes):
    '''Performs SHA256(SHA256(in_bytes)) and returns raw digest'''
    sha256_hasher = hashlib.sha256(in_bytes)
    sha256_hasher2 = hashlib.sha256(sha256_hasher.digest())
    return sha256_hasher2.digest()
    
    
def hash160_hex(in_bytes):
    '''Performs RIPEMD160(SHA256(in_bytes)) and returns HEX digest'''
    return binascii.hexlify(hash160(in_bytes)).decode('utf-8')

In [67]:
# https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses
# https://github.com/warner/python-ecdsa
# https://docs.python.org/3/library/hashlib.html
# Terminal: openssl ecparam -list_curves | grep -i secp256k1

from ecdsa import SigningKey, SECP256k1


def pub_key_from_priv_key(priv_key_hex):
    '''Obtain public key from the private key'''
    secexp = int(priv_key_hex, 16)
    signing_key = SigningKey.from_secret_exponent(secexp, curve=SECP256k1)
    vk = signing_key.get_verifying_key()
    return vk.to_string()


def bitcoin_addr_from_priv_key(priv_key_hex):
    '''Create Bitcoin address from the private key'''
    # Step 0: Having a private ECDSA key
    pub_key_raw = pub_key_from_priv_key(priv_key_hex)
    return bitcoin_addr_from_pub_key(pub_key_raw)


def get_full_pub_key(pub_key_raw):
    '''Represent public key in full/uncompressed format'''
    # (65 bytes, 1 byte 0x04, 32 bytes corresponding to X coordinate,
    # 32 bytes corresponding to Y coordinate)
    assert len(pub_key_raw) == 64
    full_pub_key = b'\x04' + pub_key_raw
    assert len(full_pub_key) == 65
    return full_pub_key


def compressed_pub_key(pub_key_raw):
    '''Represent public key in the compressed format'''
    assert len(pub_key_raw) == 64
    p_x = pub_key_raw[:32]
    p_y = pub_key_raw[32:]
    p_y_num = int(p_y)
    prefix = b'\x03' if p_y_num % 2 else b'\x02'
    compressed_pub_key = prefix + p_x
    assert len(compressed_pub_key) == 33
    return compressed_pub_key


def bitcoin_addr_from_pub_key(pub_key_raw):
    '''Create Bitcoin address from the public key'''
    # Step 1: Take the corresponding public key generated with it
    repr_pub_key = get_full_pub_key(pub_key_raw)

    # Step 2: Perform SHA-256 hashing on the public key
    # Step 3: Perform RIPEMD-160 hashing on the result of SHA-256
    vk_hash160 = hash160(repr_pub_key)

    # Step 4: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
    mainnet_format = b'\x00' + vk_hash160

    # Step 5: Perform SHA-256 hash on the extended RIPEMD-160 result
    # Step 6: Perform SHA-256 hash on the result of the previous SHA-256 hash
    full_checksum = hash256(mainnet_format)

    # Step 7: Take the first 4 bytes of the second SHA-256 hash. This is the address checksum
    checksum_4bytes = full_checksum[:4]

    # Step 8: Add the 4 checksum bytes from stage 7 at the end of
    # extended RIPEMD-160 hash from stage 4.
    # This is the 25-byte binary Bitcoin Address.
    bitcoin_addr_raw = mainnet_format + checksum_4bytes
    assert len(bitcoin_addr_raw) == 25

    # Step 9: Convert the result from a byte string into a base58 string using Base58Check encoding.
    # This is the most commonly used Bitcoin Address format
    bitcoin_addr = base256_to_base58(bitcoin_addr_raw)
    bitcoin_addr_raw_again = base58_to_base256(bitcoin_addr)
    assert bitcoin_addr_raw_again == bitcoin_addr_raw
    return bitcoin_addr


def verify_bitcoin_addr(bitcoin_addr):
    '''Verify Bitcoin address'''
    bitcoin_addr_raw = base58_to_base256(bitcoin_addr)
    assert len(bitcoin_addr_raw) == 25
    mainnet_format, checksum_4bytes = bitcoin_addr_raw[:21], bitcoin_addr_raw[21:]

    # Verify that the double SHA-256 has the same prefix as checksum_4bytes
    full_checksum = hash256(mainnet_format)
    assert checksum_4bytes == full_checksum[:4]

    return True

In [70]:
TEST_CASES = [
    {
        'priv': '18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725',
        'addr': '16UwLL9Risc3QfPqBUvKofHmBQ7wMtjvM',
    },
]

'''
{
    'priv': '80B1D685BAE45AF1D33231F47A8BC6F54CECC7DE76C15D60D658B38603346BD6910193D04B08',
    'addr': '1D2Gme2513ncWsxB4DchzT3ukeNUXYVv3c',
},
'''

for test_case in TEST_CASES:
    print('Verifying the case: ', test_case)
    priv_key_hex = test_case['priv']
    bitcoin_addr = bitcoin_addr_from_priv_key(priv_key_hex)
    assert bitcoin_addr == test_case['addr']
    assert verify_bitcoin_addr(bitcoin_addr)
    print('This case was successful!')

Verifying the case:  {'priv': '18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725', 'addr': '16UwLL9Risc3QfPqBUvKofHmBQ7wMtjvM'}
This case was successful!


In [69]:
base256_to_base58(unhexlify('80B1D685BAE45AF1D33231F47A8BC6F54CECC7DE76C15D60D658B38603346BD6910193D04B08'))

'L3BQRZyUzNPUPbt1HtGby9UwVb5iz2RyEk9jQk1vqhnL5CwFZHiX'